In [89]:
import os
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.corpora import Dictionary


In [90]:
def preprocess(docs,no_below=20,no_above=0.7):
    # input is a an array of docs; each is one string
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]
    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]   

    # Lemmatize all words in documents.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    
    # Delete words based on their frequency in the whole corps
    # Create a dictionary representation of the documents.
    dictionary = Dictionary(docs)
    #set_trace()
    # Filter out words that occur less than 20 documents, or more than 70% of the documents.
    dictionary.filter_extremes(no_below, no_above)
    
    # According to the filtered dictionary, reconstruct the corpus
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    return corpus, dictionary
    # TODO: linebreaks should be recovered into single word? search for solution first


In [91]:
years = list(range(22, 40))
dirs = ['text_data/volume_{}/'.format(y) for y in years]

In [92]:
ndocs = []
docs = []
import codecs


for d in dirs:
    fnames = os.listdir(d)
    # start with 1/10 of the data
    #fnames = [t[1] for t in enumerate(fnames) if t[0] % 10 == 0]
    for fn in fnames:
        with open(d + fn, 'r', encoding='utf-8') as f:
            docs.append(f.read())
    ndocs.append(len(fnames))

In [93]:
corpus, dictionary = preprocess(docs,no_below=18,no_above=0.5)

In [96]:

import logging
from gensim.models import ldaseqmodel
from gensim.models.wrappers.dtmmodel import DtmModel

from gensim.corpora import Dictionary, bleicorpus, textcorpus
import numpy as np
from gensim.matutils import hellinger
import time


In [ ]:
t0= time.time()
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

dtm_path = "/Users/alex/dtm/bin/dtm-darwin64"
model = DtmModel(dtm_path, corpus=corpus, time_slices=ndocs, num_topics=20, id2word=dictionary, initialize_lda=True)

dt = time.time() - t0


In [135]:
yeardata = []
for t in range(len(ndocs)):
    yeardata.append(model.dtm_vis(corpus, time=t))

In [136]:
import pickle
pickle.dump(yeardata, open( "output/20topics.p", "wb" ))

In [137]:
yeardata[0][1].shape

(20, 19634)

In [138]:
import pyLDAvis


In [141]:
y = 0
vis_dtm = pyLDAvis.prepare(topic_term_dists=yeardata[y][1], 
                           doc_topic_dists=yeardata[y][0], 
                           doc_lengths=yeardata[y][2], 
                           vocab=yeardata[y][4], 
                           term_frequency=yeardata[y][3])



/Users/alex/anaconda/envs/py3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [143]:
#pyLDAvis.display(vis_dtm)


In [134]:
dt

9196.018929243088